In [9]:
# ruff: noqa: F401

In [10]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


In [11]:
import sys

from dotenv import load_dotenv

sys.path.insert(0, "..")

load_dotenv()

True

In [12]:
from datetime import datetime, timedelta
from pathlib import Path

import duckdb
import hvplot.polars
import numpy as np
import polars as pl

In [13]:
from tsdb_benchmarks.settings import SETTINGS

In [14]:
db = duckdb.connect(SETTINGS.results_directory / "results.db")

In [16]:
db.execute("select * from benchmark").pl()

id,name,operation,started_at,finished_at,deleted_at,notes
i32,str,str,datetime[μs],datetime[μs],datetime[μs],str
1,"""monetdb""","""populate""",2025-07-17 11:42:45.534279,null,null,null
2,"""monetdb""","""populate""",2025-07-17 11:42:59.937159,null,null,null
3,"""monetdb""","""populate""",2025-07-17 11:43:23.434178,null,null,null
4,"""monetdb""","""populate""",2025-07-17 12:10:19.207872,null,null,null
5,"""monetdb""","""populate""",2025-07-17 12:19:23.157480,null,null,null
6,"""monetdb""","""populate""",2025-07-17 12:20:06.959376,2025-07-17 12:20:07.113180,null,null
7,"""monetdb""","""populate""",2025-07-17 12:20:19.711093,2025-07-17 12:20:19.863833,null,null
8,"""monetdb""","""populate""",2025-07-17 12:20:26.884675,null,null,null
9,"""monetdb""","""populate""",2025-07-17 12:21:57.249120,2025-07-17 12:32:45.608451,null,null


In [19]:
df = db.execute("select * from metric where benchmark_id = 9 order by time").pl()

df

benchmark_id,time,cpu_percent,mem_mb,disk_mb
i32,datetime[μs],f32,i32,i32
9,2025-07-17 12:21:57.404923,29.337234,91,39
9,2025-07-17 12:21:58.600301,23.71715,324,271
9,2025-07-17 12:22:00.445503,75.242035,1411,1760
9,2025-07-17 12:22:02.454210,99.80619,4132,2988
9,2025-07-17 12:22:04.452985,98.113945,6731,5793
…,…,…,…,…
9,2025-07-17 12:32:35.216730,271.706573,7467,14192
9,2025-07-17 12:32:37.196828,99.011024,9613,14192
9,2025-07-17 12:32:39.413532,100.149429,11307,14885


In [21]:
db.execute("select * from event where benchmark_id = 9 order by time").pl()

benchmark_id,time,name,type
i32,datetime[μs],str,cat
9,2025-07-17 12:21:57.844117,"""schema""","""start"""
9,2025-07-17 12:21:57.869810,"""schema""","""end"""
9,2025-07-17 12:21:57.882884,"""insert_customers""","""start"""
9,2025-07-17 12:21:57.923984,"""insert_customers""","""end"""
9,2025-07-17 12:21:57.935552,"""insert_products""","""start"""
…,…,…,…
9,2025-07-17 12:21:59.258010,"""insert_orders""","""end"""
9,2025-07-17 12:21:59.336234,"""insert_order_items""","""start"""
9,2025-07-17 12:22:14.611693,"""insert_order_items""","""end"""


In [20]:
df.select("time", "mem_mb", "disk_mb").hvplot(x="time", grid=True) + df.select("time", "cpu_percent").hvplot(
    x="time", grid=True
)

:Layout
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [time]   (value)
   .Curve.I     :Curve   [time]   (cpu_percent)